Notebook to Explore Text Features

The data used was created in the regressions notebook under the section dated 4/10/18 i.e. it is three dataframes of listings with average price for each

In [1]:
#starting with listing descriptions

In [17]:
import pandas as pd

In [18]:
listings = pd.read_csv('Datasources/inside_airbnb/listings.csv')

In [4]:
listings.columns

Index([u'id', u'listing_url', u'scrape_id', u'last_scraped', u'name',
       u'summary', u'space', u'description', u'experiences_offered',
       u'neighborhood_overview', u'notes', u'transit', u'access',
       u'interaction', u'house_rules', u'thumbnail_url', u'medium_url',
       u'picture_url', u'xl_picture_url', u'host_id', u'host_url',
       u'host_name', u'host_since', u'host_location', u'host_about',
       u'host_response_time', u'host_response_rate', u'host_acceptance_rate',
       u'host_is_superhost', u'host_thumbnail_url', u'host_picture_url',
       u'host_neighbourhood', u'host_listings_count',
       u'host_total_listings_count', u'host_verifications',
       u'host_has_profile_pic', u'host_identity_verified', u'street',
       u'neighbourhood', u'neighbourhood_cleansed',
       u'neighbourhood_group_cleansed', u'city', u'state', u'zipcode',
       u'market', u'smart_location', u'country_code', u'country', u'latitude',
       u'longitude', u'is_location_exact', u'prope

#Text columns:
space', u'description', u'experiences_offered',
       u'neighborhood_overview', u'notes', u'transit', u'access',
       u'interaction', u'house_rules'

In [5]:
#some basic data exploration to see what can be done

In [6]:
text_features = ['space', 'description', 'experiences_offered', 'neighborhood_overview', 'notes', 'transit', 'access', 'interaction', 'house_rules']

In [7]:
len(listings)

6608

In [8]:
#How many rows are null
for i in text_features:
    print("Column Name: %s, Nulls Count: %i," %(i, listings[i].isnull().sum()))

Column Name: space, Nulls Count: 1635,
Column Name: description, Nulls Count: 2,
Column Name: experiences_offered, Nulls Count: 0,
Column Name: neighborhood_overview, Nulls Count: 2471,
Column Name: notes, Nulls Count: 3375,
Column Name: transit, Nulls Count: 2636,
Column Name: access, Nulls Count: 2273,
Column Name: interaction, Nulls Count: 2530,
Column Name: house_rules, Nulls Count: 1642,


In [9]:
listings['description'].head()

0    Aquatica Waterpark, Sleep train Amphitheater, ...
1    Your spacious room awaiting is with a Queen Si...
2    This is an immaculate 3 bedroom, 2 1/2 bath co...
3    This 2 Story TownHome  is close to Otay Ranch ...
4    Hello; we are offering a private secluded bedr...
Name: description, dtype: object

In [5]:
import nltk
import string
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [23]:
def tokenize(text):
    stem = nltk.stem.SnowballStemmer('english')
    text = text.lower()

    for token in nltk.word_tokenize(text):
        if token in string.punctuation: continue
        yield stem.stem(token)

In [24]:
def vectorize(doc):
    features = defaultdict(int)

    for token in tokenize(doc):
        features[token] += 1

    return features

In [25]:
vectors = map(vectorize, listings['description'].iloc[:1])

In [27]:
vectorizer = CountVectorizer()
vectors = vectorizer.fit_transform(listings['description'].iloc[:1])

In [28]:
vectors

<1x42 sparse matrix of type '<type 'numpy.int64'>'
	with 42 stored elements in Compressed Sparse Row format>

In [30]:
tfidf  = TfidfVectorizer()
corpus = tfidf.fit_transform(listings['description'].iloc[:1])

In [32]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

In [34]:
"""import unicodedata
from sklearn.base import BaseEstimator, TransformerMixin


class TextNormalizer(BaseEstimator, TransformerMixin):

    def __init__(self, language='english'):
        self.stopwords  = set(nltk.corpus.stopwords.words(language))
        self.lemmatizer = WordNetLemmatizer()

    def is_punct(self, token):
        return all(
            unicodedata.category(char).startswith('P') for char in token
        )

    def is_stopword(self, token):
        return token.lower() in self.stopwords"""

In [40]:
model = Pipeline([
            #('norm', TextNormalizer()),
            ('tfidf', CountVectorizer(
                                      preprocessor=None, lowercase=False)),
            ('model', LatentDirichletAllocation(n_components=3)),
        ])

In [41]:
model.fit_transform(listings['description'].iloc[:1])

/Users/pauldefusco/anaconda2/envs/py27/lib/python2.7/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


array([[ 0.00630038,  0.00630097,  0.98739865]])

In [45]:
model.steps[-1][1].components_

array([[ 0.48526874,  0.46780674,  0.4506659 ,  0.43849857,  0.47683737,
         0.49175364,  0.46894963,  0.48306749,  0.48838241,  0.42804966,
         0.43723167,  0.46663385,  0.46083524,  0.45270162,  0.47156109,
         0.46384839,  0.43383636,  0.46544648,  0.47114012,  0.48056441,
         0.43012196,  0.47679485,  0.43738697,  0.47024862,  0.46610585,
         0.48231129,  0.44705251,  0.48191674,  0.45480082,  0.45676223,
         0.4606544 ,  0.45713291,  0.47130262,  0.4931026 ,  0.4893857 ,
         0.42219724,  0.44698712,  0.50010835,  0.47749707,  0.48875821,
         0.47143302,  0.45563601,  0.45106472],
       [ 0.50841405,  0.44805375,  0.47656416,  0.45819918,  0.48598188,
         0.44549965,  0.49490127,  0.45038721,  0.47362984,  0.50362255,
         0.50656658,  0.44533206,  0.46256918,  0.44681533,  0.48328912,
         0.43283261,  0.48099818,  0.44830807,  0.46054512,  0.44190069,
         0.50526221,  0.46001181,  0.44778226,  0.44501719,  0.47126264,
   

In [50]:
lst = []
for i in model.steps[-1][1].components_:
    f=i.argsort()[:-(2 - 1): -1]
    lst.append(f)
    print f
    

[]
[]
[]


In [49]:
lst

[array([], dtype=int64), array([], dtype=int64), array([], dtype=int64)]

In [1]:
##Recreating what I had done before losing everything

In [20]:
listings['description'].dtypes

dtype('O')

In [22]:
listings['description'].isnull().sum()

2

In [23]:
corpus = listings['description'].fillna('none')

In [24]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             # max_features=50000,             # max number of uniq words
                            )

In [25]:
data_vectorized = vectorizer.fit_transform(corpus)

In [27]:
from sklearn.decomposition import LatentDirichletAllocation

In [28]:
lda_model = LatentDirichletAllocation(n_topics=20,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

/Users/pauldefusco/anaconda2/envs/py27/lib/python2.7/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=10, n_jobs=-1, n_topics=20, perp_tol=0.1,
             random_state=100, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)


In [32]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))

# See model parameters
print(lda_model.get_params())

('Log Likelihood: ', -3209062.0345833665)
('Perplexity: ', 849.87164800130677)
{'learning_offset': 10.0, 'n_jobs': -1, 'topic_word_prior': None, 'perp_tol': 0.1, 'evaluate_every': -1, 'max_iter': 10, 'mean_change_tol': 0.001, 'batch_size': 128, 'max_doc_update_iter': 100, 'learning_decay': 0.7, 'n_components': 10, 'random_state': 100, 'doc_topic_prior': None, 'n_topics': 20, 'total_samples': 1000000.0, 'learning_method': 'online', 'verbose': 0}


In [30]:
"""# Define Search Param
search_params = {'n_components': [10, 15, 20, 25, 30], 'learning_decay': [.5, .7, .9]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(data_vectorized)"""

"# Define Search Param\nsearch_params = {'n_components': [10, 15, 20, 25, 30], 'learning_decay': [.5, .7, .9]}\n\n# Init the Model\nlda = LatentDirichletAllocation()\n\n# Init Grid Search Class\nmodel = GridSearchCV(lda, param_grid=search_params)\n\n# Do the Grid Search\nmodel.fit(data_vectorized)"

In [34]:
"""# Best Model - using original lda_mdoel for now
best_lda_model = model.best_estimator_
best_lda_model = lda_model
# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))"""

'# Best Model - using original lda_mdoel for now\nbest_lda_model = model.best_estimator_\nbest_lda_model = lda_model\n# Model Parameters\nprint("Best Model\'s Params: ", model.best_params_)\n\n# Log Likelihood Score\nprint("Best Log Likelihood Score: ", model.best_score_)\n\n# Perplexity\nprint("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))'

In [37]:
import numpy as np

In [38]:
# Create Document - Topic Matrix
lda_output = lda_model.transform(data_vectorized)

# column names
topicnames = ["Topic" + str(i) for i in range(lda_model.n_topics)]

# index names
docnames = ["Doc" + str(i) for i in range(len(corpus))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

In [40]:
df_document_topic.head()

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,...,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,Topic19,dominant_topic
Doc0,0.00,0.0,0.00,0.87,0.00,0.00,0.0,0.00,0.00,0.00,...,0.0,0.00,0.0,0.00,0.0,0.04,0.00,0.00,0.0,3
Doc1,0.05,0.0,0.00,0.29,0.00,0.18,0.0,0.00,0.08,0.16,...,0.0,0.00,0.0,0.00,0.0,0.14,0.08,0.00,0.0,3
Doc2,0.00,0.0,0.32,0.25,0.00,0.00,0.0,0.00,0.00,0.00,...,0.0,0.00,0.0,0.00,0.0,0.00,0.10,0.00,0.0,2
Doc3,0.00,0.0,0.05,0.80,0.00,0.00,0.0,0.00,0.00,0.00,...,0.0,0.00,0.0,0.08,0.0,0.00,0.00,0.00,0.0,3
Doc4,0.00,0.0,0.00,0.59,0.01,0.20,0.0,0.01,0.00,0.07,...,0.0,0.02,0.0,0.00,0.0,0.00,0.00,0.01,0.0,3


In [45]:
df_document_topic.index = [i for i in range(len(df_document_topic))]

In [46]:
out = df_document_topic.merge(listings, left_index=True, right_index=True)

In [48]:
out.head()

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,0.00,0.0,0.00,0.87,0.00,0.00,0.0,0.00,0.00,0.00,...,10.0,f,NaN,"SAN DIEGO, SAN DIEGO TOURISM MARKETING DISTRIC...",f,moderate,f,f,2,4.57
1,0.05,0.0,0.00,0.29,0.00,0.18,0.0,0.00,0.08,0.16,...,10.0,f,NaN,NaN,f,strict,f,f,1,0.76
2,0.00,0.0,0.32,0.25,0.00,0.00,0.0,0.00,0.00,0.00,...,8.0,f,NaN,NaN,f,flexible,f,f,1,0.09
3,0.00,0.0,0.05,0.80,0.00,0.00,0.0,0.00,0.00,0.00,...,10.0,f,NaN,NaN,f,moderate,f,f,1,3.00
4,0.00,0.0,0.00,0.59,0.01,0.20,0.0,0.01,0.00,0.07,...,NaN,f,NaN,NaN,f,strict,f,f,2,NaN


In [47]:
out.to_csv('listings_withtopics.csv')